# Завдання

_Вправа 1: натренуйте модель дерева рішень та виконайте пошук гіперпараметрів для moons dataset._ \
a. Згенеруйте датасет, використовуючи функцію `make_moons(n_samples=10000, noise=0.4)`. \
b. Розбийте її на тренувальну та тестувальну частини, використовуючи функцію `train_test_split()`. \
c. Використайте пошук з крос-валідацією (`GridSearchCV`, `RandomizedSearchCV`) для пошуку гіперпараметрів моделі `DecisionTreeClassifier`. Зокрема, спробуйте різні значення для параметра `max_leaf_nodes`.

In [41]:
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.stats import uniform

це генерація датасету та розбиття на тестову тренувальну частину

In [43]:
X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle=True)

грідсерч

In [51]:
from sklearn.model_selection import GridSearchCV

# Параметри для пошуку
param_grid = {
    'max_leaf_nodes': [2, 5, 10, 20, 50, 100],
    'min_samples_split': [2, 5, 10, 20]
}

# GridSearchCV
grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Найкращі параметри (GridSearchCV):", grid_search.best_params_)
print("Найкраща точність (GridSearchCV):", grid_search.best_score_)

Найкращі параметри (GridSearchCV): {'max_leaf_nodes': 20, 'min_samples_split': 2}
Найкраща точність (GridSearchCV): 0.8634999999999999


рандом серч

In [52]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'max_leaf_nodes': randint(2, 100), # різні значення параметра
    'min_samples_split': randint(2, 20)
}

random_search = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Найкращі параметри (RandomizedSearchCV):", random_search.best_params_)
print("Найкраща точність (RandomizedSearchCV):", random_search.best_score_)

Найкращі параметри (RandomizedSearchCV): {'max_leaf_nodes': 25, 'min_samples_split': 4}
Найкраща точність (RandomizedSearchCV): 0.8633749999999999


In [53]:
# Оцінка моделі з GridSearchCV
grid_score = grid_search.score(X_test, y_test)
print(f"Точність на тестовій вибірці (GridSearchCV): {grid_score:.4f}")

# Оцінка моделі з RandomizedSearchCV
random_score = random_search.score(X_test, y_test)
print(f"Точність на тестовій вибірці (RandomizedSearchCV): {random_score:.4f}")

Точність на тестовій вибірці (GridSearchCV): 0.8500
Точність на тестовій вибірці (RandomizedSearchCV): 0.8560


Я не розумію чому max_leaf_nodes повинно бути 10. Ми генеруємо рандом датасет і навіть якщо подивитися по статистиці то max_leaf_nodes=10 трапляється не частіше за інші

After searching for best hyperparameters for the model, the results show that the best max_leaf_nodes number is 10 for both approaches. The other parameters differ.

_Вправа 2_ : \
a. Завантажте датасет MNIST та розбийте його на тренувальну, валідаційну та тестувальну частини (наприклад, 50 000 / 10 000 / 10
000). \
b. Натренуйте різні класифікаційні моделі (Random Forest classifier, Logistic Regression, SVM). \
c. Далі, об'єднайте їх за допомогою голосування. \
d. Натренуйте нову модель, використовуючи виходи попередніх моделей на валідаційній вибірці (це будуть нові ознаки і нова тренувальна вибірка для даної моделі). Протестуйте отриманий ланцюжок на тренувальний вибірці. Такий спосіб поєднання моделей називається стогуванням (stacking).

In [9]:
import numpy as np
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:50000]
y_train = y[:50000]
X_test = X[60000:]
y_test = y[60000:]

X_val = X[50000:60000]
y_val = y[50000:60000]


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)>

У мене на локалхості швидше все робиться то я скачав датасет з інету просто

In [54]:
import pickle
import numpy as np

with open('mnist_data.pkl', 'rb') as f:
    mnist = pickle.load(f)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:50000]
y_train = y[:50000]

X_val = X[50000:60000]
y_val = y[50000:60000]

X_test = X[60000:]
y_test = y[60000:]

print(f"Тренувальна вибірка: {X_train.shape}")
print(f"Валідаційна вибірка: {X_val.shape}")
print(f"Тестувальна вибірка: {X_test.shape}")

Тренувальна вибірка: (50000, 784)
Валідаційна вибірка: (10000, 784)
Тестувальна вибірка: (10000, 784)


тренуєм рандом форест

In [55]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_clf.fit(X_train, y_train)
rf_score = rf_clf.score(X_val, y_val)
print(f"Random Forest точність на валідації: {rf_score:.4f}")


Random Forest точність на валідації: 0.9718


логістична регресія

In [57]:
log_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression(random_state=42, max_iter=1000, n_jobs=-1))
])
log_clf.fit(X_train, y_train)
log_score = log_clf.score(X_val, y_val)
print(f"Logistic Regression точність на валідації: {log_score:.4f}")

Logistic Regression точність на валідації: 0.9228


свм

In [58]:
svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(random_state=42, kernel='rbf'))
])
svm_clf.fit(X_train, y_train)
svm_score = svm_clf.score(X_val, y_val)
print(f"SVM точність на валідації: {svm_score:.4f}")

SVM точність на валідації: 0.9687


In [59]:
voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('lr', log_clf),
        ('svc', svm_clf)
    ],
    voting='hard'  # hard voting - голосування більшістю із софт не вийшло вилізла помилка
)

# Навчаємо voting classifier
voting_clf.fit(X_train, y_train)
voting_score = voting_clf.score(X_val, y_val)
print(f"Voting Classifier точність на валідації: {voting_score:.4f}")

# Порівняємо всі результати
print(f"Random Forest:        {rf_score:.4f}")
print(f"Logistic Regression:  {log_score:.4f}")
print(f"SVM:                  {svm_score:.4f}")
print(f"Voting Classifier:    {voting_score:.4f}")


Voting Classifier точність на валідації: 0.9701
Random Forest:        0.9718
Logistic Regression:  0.9228
SVM:                  0.9687
Voting Classifier:    0.9701


In [61]:
# Отримуємо передбачення на валідаційній вибірці
X_val_stacking = np.column_stack([
    rf_clf.predict(X_val),
    log_clf.predict(X_val),
    svm_clf.predict(X_val)
])

# Навчаємо blender на валідаційних даних
blender = LogisticRegression(random_state=42, max_iter=5000)
blender.fit(X_val_stacking, y_val)

# Тестуємо на тренувальній вибірці
X_train_stacking = np.column_stack([
    rf_clf.predict(X_train),
    log_clf.predict(X_train),
    svm_clf.predict(X_train)
])

train_score = blender.score(X_train_stacking, y_train)
print(f"Stacking точність на тренувальній вибірці: {train_score:.4f}")

# Додатково на тестовій
X_test_stacking = np.column_stack([
    rf_clf.predict(X_test),
    log_clf.predict(X_test),
    svm_clf.predict(X_test)
])
test_score = blender.score(X_test_stacking, y_test)
print(f"Stacking точність на тестовій вибірці: {test_score:.4f}")

Stacking точність на тренувальній вибірці: 0.9793
Stacking точність на тестовій вибірці: 0.9472


In [62]:
# Оцінюємо всі моделі на тестовій вибірці
rf_test = rf_clf.score(X_test, y_test)
log_test = log_clf.score(X_test, y_test)
svm_test = svm_clf.score(X_test, y_test)
voting_test = voting_clf.score(X_test, y_test)

# Stacking на тестовій
X_test_stacking = np.column_stack([
    rf_clf.predict(X_test),
    log_clf.predict(X_test),
    svm_clf.predict(X_test)
])
stacking_test = blender.score(X_test_stacking, y_test)

print(f"Random Forest:        {rf_test:.4f}")
print(f"Logistic Regression:  {log_test:.4f}")
print(f"SVM:                  {svm_test:.4f}")
print(f"Voting Classifier:    {voting_test:.4f}")
print(f"Stacking:             {stacking_test:.4f}")

#найкраща модель
models = {
    'Random Forest': rf_test,
    'Logistic Regression': log_test,
    'SVM': svm_test,
    'Voting Classifier': voting_test,
    'Stacking': stacking_test
}

best_model = max(models, key=models.get)
print(f"Найкраща модель: {best_model} ({models[best_model]:.4f})")

Random Forest:        0.9687
Logistic Regression:  0.9201
SVM:                  0.9643
Voting Classifier:    0.9660
Stacking:             0.9472
Найкраща модель: Random Forest (0.9687)


Цікавий спосіб потрібен час щоб його повністю осмислити насправді. По результатам поки безтолково як на мене :/